# Introduction

### Store Sales - Time Series Forcasting
Aim is predicting sales for each product family and store combinations. There are 6 datasets given.
1. Train
2. Test
3. Store
4. Transactions
5. Holidays and Events
6. Daily Oil Price

### import necessary modules

In [1]:
#from pivottablejs import pivot_ui

import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
import gc

import seaborn as sns
sns.set(style = 'whitegrid', color_codes = True)
%matplotlib inline

#For statistical tests
import scipy.stats as st

#For formula notation (similar to R)
import statsmodels.formula.api as smf

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import operator

### loading data

In [2]:
train=pd.read_csv(r"C:\Users\Nathaniel Havim\Desktop\Azubi\LP2_DAP\train.csv")
test=pd.read_csv(r"C:\Users\Nathaniel Havim\Desktop\Azubi\LP2_DAP\test.csv")
oil=pd.read_csv(r"C:\Users\Nathaniel Havim\Desktop\Azubi\LP2_DAP\oil.csv")
holiday=pd.read_csv(r"C:\Users\Nathaniel Havim\Desktop\Azubi\LP2_DAP\holidays_events.csv")
store=pd.read_csv(r"C:\Users\Nathaniel Havim\Desktop\Azubi\LP2_DAP\stores.csv")
trans=pd.read_csv(r"C:\Users\Nathaniel Havim\Desktop\Azubi\LP2_DAP\transactions.csv")
sample=pd.read_csv(r"C:\Users\Nathaniel Havim\Desktop\Azubi\LP2_DAP\sample_submission.csv")

## Exploratoty Data Anlysis(EDA)

### train data

The training data, comprising time series of features store_nbr, family, 
    and onpromotion as well as the target sales.
    
- **store_nbr** identifies the store at which the products are sold.

-   **family** identifies the type of product sold.

-   **sales** gives the total sales for a product family at a particular store
    at a given date. Fractional values are possible since products can be sold in 
    fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).

-   **onpromotion** gives the total number of items in a product family that
    were being promoted at a store at a given date.

In [3]:
# Calling train data
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [5]:
# Change date to datetime format
train['date']=pd.to_datetime(train['date'])
train['date']

0         2013-01-01
1         2013-01-01
2         2013-01-01
3         2013-01-01
4         2013-01-01
             ...    
3000883   2017-08-15
3000884   2017-08-15
3000885   2017-08-15
3000886   2017-08-15
3000887   2017-08-15
Name: date, Length: 3000888, dtype: datetime64[ns]

In [6]:
# Set date to index
train=train.set_index('date')

In [7]:
train

,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.000,0
2013-01-01,1,1,BABY CARE,0.000,0
2013-01-01,2,1,BEAUTY,0.000,0
2013-01-01,3,1,BEVERAGES,0.000,0
2013-01-01,4,1,BOOKS,0.000,0
...,...,...,...,...,...
2017-08-15,3000883,9,POULTRY,438.133,0
2017-08-15,3000884,9,PREPARED FOODS,154.553,1
2017-08-15,3000885,9,PRODUCE,2419.729,148


### Test data

-   The test data, having the same features as the training data. You will predict the target sales for the dates in this file.

-   The dates in the test data are for the 15 days after the last date in the training data.

In [8]:
# Calling test data
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


In [10]:
# Change data column in test data to datetime format
test['date']=pd.to_datetime(test['date'])

In [11]:
test=test.set_index('date')

In [12]:
test

,id,store_nbr,family,onpromotion
date,,,,
2017-08-16,3000888,1,AUTOMOTIVE,0
2017-08-16,3000889,1,BABY CARE,0
2017-08-16,3000890,1,BEAUTY,2
2017-08-16,3000891,1,BEVERAGES,20
2017-08-16,3000892,1,BOOKS,0
...,...,...,...,...
2017-08-31,3029395,9,POULTRY,1
2017-08-31,3029396,9,PREPARED FOODS,0
2017-08-31,3029397,9,PRODUCE,1


### oil data

-   **Daily oil price** which includes values during both the train and
     test data timeframes. (Ecuador is an oil-dependent country and its
     economical health is highly vulnerable to shocks in oil prices.)

In [13]:
# calling the oil data
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [14]:
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [15]:
oil['date']=pd.to_datetime(oil['date'])

In [16]:
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [17]:
# set index to date
oil=oil.set_index('date')

### holiday_event data

-   Holidays and Events, with metadata

> **NOTE**: Pay special attention to the transferred column. A holiday
> that is transferred officially falls on that calendar day but was
> moved to another date by the government. A transferred day is more
> like a normal day than a holiday. To find the day that it was
> celebrated, look for the corresponding row where type is **Transfer**.

In [18]:
# calling the holiday_event data
holiday.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [19]:
holiday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [20]:
holiday['date']=pd.to_datetime(holiday['date'])

In [21]:
holiday.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


### store data


-   Store metadata, including city, state, type, and cluster.

-   cluster is a grouping of similar stores.

In [22]:
#calling stores data 
store.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [23]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


### trans (transactions) data

-   Contains date, store_nbr and transaction made on that specific date.

In [24]:
trans.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [25]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [26]:
trans['date']=pd.to_datetime(trans['date'])

In [27]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


In [28]:
trans=trans.set_index('date')

### sample data

-   A sample submission file in the correct format.
-   this is a guide on how the final file for submission should look like

In [29]:
sample.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


# Data Preparation

#### Checking for the persentage of missing values in data

In [30]:
train_nan=(train.isna().sum() / train.shape[0])*100
train_nan

id             0.0
store_nbr      0.0
family         0.0
sales          0.0
onpromotion    0.0
dtype: float64

There no missing values in train data

In [31]:
oil_nan=(oil.isna().sum() / oil.shape[0])*100
oil_nan

dcoilwtico    3.530378
dtype: float64

There are 3.5% missing values in oil data in the dcoilwtico column

In [32]:
# interpolating missing values 
oil['dcoilwtico'].interpolate(method='linear', inplace=True)

# show the trend in oil data

In [33]:
store_nan=(store.isna().sum() / store.shape[0])*100
store_nan

store_nbr    0.0
city         0.0
state        0.0
type         0.0
cluster      0.0
dtype: float64

There are no missing values in store data

## Hypotheses

**1. Null Hypothesis (H0)**: Season has no direct effect on total sales.

**Alternative Hypothesis (H1)**: Season has direct effect on total sales.

**2. Null Hypothesis (H0)**: Change in oil price has no significant change in total sales

**Alternative Hypothesis (H1)**: Change oil price singnificantly affects total sales

**3. Null Hypothesis (H0)**: Promotion does not positively affects total sales

**Alternative Hypothesis (H1)**: Promotion positively affects total sales

## Questions

1. Is the train dataset complete (has all the required dates)?

2. Which dates have the lowest and highest sales for each year?

3. Did the earthquake impact sales?

4. Are certain groups of stores selling more products? (Cluster, city, state, type)

5. Are sales affected by promotions, oil prices and holidays?

6. What analysis can we get from the date and its extractable features?

7. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

8. What are the daily sales of each store?

9. Does the dates for paying public sectors affect sales?

10. How does seasonal occasions affect sales?

### Merge necessary datasets

In [34]:
# merge train and trans
df = pd.merge(train, trans, how='outer', on=['date', 'store_nbr'])

In [35]:
df1 = pd.merge(df, holiday, how='outer', on='date')

In [36]:
df2 = pd.merge(df1, oil, how='outer', on='date')

In [37]:
df3 = pd.merge(df2, store, how='outer', on='store_nbr')

In [38]:
df3.head()

,date,id,store_nbr,family,sales,onpromotion,transactions,type_x,locale,locale_name,description,transferred,dcoilwtico,city,state,type_y,cluster
0,2013-01-01,0.0,1.0,AUTOMOTIVE,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13.0
1,2013-01-01,1.0,1.0,BABY CARE,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13.0
2,2013-01-01,2.0,1.0,BEAUTY,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13.0
3,2013-01-01,3.0,1.0,BEVERAGES,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13.0
4,2013-01-01,4.0,1.0,BOOKS,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13.0


In [39]:
# checking description of merged data
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3054427 entries, 0 to 3054426
Data columns (total 17 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   id            float64       
 2   store_nbr     float64       
 3   family        object        
 4   sales         float64       
 5   onpromotion   float64       
 6   transactions  float64       
 7   type_x        object        
 8   locale        object        
 9   locale_name   object        
 10  description   object        
 11  transferred   object        
 12  dcoilwtico    float64       
 13  city          object        
 14  state         object        
 15  type_y        object        
 16  cluster       float64       
dtypes: datetime64[ns](1), float64(7), object(9)
memory usage: 419.5+ MB


##### Set date as index for the merged data

In [40]:
df3.set_index('date')

,id,store_nbr,family,sales,onpromotion,transactions,type_x,locale,locale_name,description,transferred,dcoilwtico,city,state,type_y,cluster
date,,,,,,,,,,,,,,,,
2013-01-01,0.0,1.0,AUTOMOTIVE,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13.0
2013-01-01,1.0,1.0,BABY CARE,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13.0
2013-01-01,2.0,1.0,BEAUTY,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13.0
2013-01-01,3.0,1.0,BEVERAGES,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13.0
2013-01-01,4.0,1.0,BOOKS,0.0,0.0,NaN,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.65,NaN,NaN,NaN,NaN
2017-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.40,NaN,NaN,NaN,NaN
2017-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.46,NaN,NaN,NaN,NaN


Notes
 1. Drop id column
 2. Rename type_x and type_y
 3. interpolate missing values
 4. Create year, month and day columns


##### Checking for missing values in the merged data

In [41]:
df3_nan=(df3.isna().sum() / df3.shape[0]) * 100

In [42]:
df3_nan

date             0.000000
id               0.002586
store_nbr        0.002586
family           0.002586
sales            0.002586
onpromotion      0.002586
transactions     8.158519
type_x          83.545457
locale          83.545457
locale_name     83.545457
description     83.545457
transferred     83.545457
dcoilwtico      28.822820
city             0.002586
state            0.002586
type_y           0.002586
cluster          0.002586
dtype: float64

There are a lot of missing values in the data after merging different datasets

In [43]:
# rename type_x: 'holiday_type' and type_y: 'store_type'
df3=df3.rename(columns={'type_x': 'holiday_type', 'type_y': 'store_type'}, inplace=True)

##### Handling missing columns


**Q1.** Is the train dataset complete (has all the required dates)?

Answering Q1: The train needs year, month and day columns and therefore it is not complete

##### Create new columns: year, month and day

In [44]:
df3['Year'] = df3['date'].dt.year
df3['Month'] = df3['date'].dt.month_name()
df3['Day'] = df3['date'].dt.day_name()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#oil['avg_oil'] = oil['dcoilwtico'].rolling(7).mean()
#calendar = calendar.join(oil.avg_oil)
#calendar['avg_oil'].fillna(method = 'ffill', inplace = True)
#calendar.dropna(inplace = True)